In [50]:
# Install necessary packages
!pip install --quiet -U langchain_openai langchain_core langgraph

# Import necessary modules
import os
import getpass

def _set_env(var: str, value: str = None):
    if value:
        os.environ[var] = value
    elif not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# Set API keys
_set_env("OPENAI_API_KEY")
_set_env("LANGCHAIN_API_KEY")

# Set LangChain tracing (optional)
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "dlp-agent"

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4", temperature=0)

# Define data models
from pydantic import BaseModel, Field
from typing import List, Optional

class Threat(BaseModel):
    indicator: str = Field(description="Indicator of compromise (IP, domain, etc.).")
    type: str = Field(description="Type of threat (e.g., malicious_ip, malicious_domain, phishing_email).")
    description: str = Field(default="", description="Description of the threat.")
    severity: str = Field(default="", description="Severity level (e.g., Low, Medium, High, Critical).")
    status: str = Field(default="Pending", description="Analysis status (Pending, Analyzed).")

class SecurityAnalyst(BaseModel):
    name: str = Field(description="Name of the security analyst.")
    role: str = Field(description="Role or job title.")
    expertise: str = Field(description="Specific expertise or specialization.")
    experience_years: int = Field(description="Number of years of experience in the field.")
    certifications: List[str] = Field(description="Relevant certifications (e.g., CISSP, CEH).")
    personality_traits: List[str] = Field(description="Personality traits that affect analysis style.")
    description: str = Field(description="Detailed description of the analyst's focus, concerns, and motives.")

    @property
    def persona(self) -> str:
        certifications = ', '.join(self.certifications)
        traits = ', '.join(self.personality_traits)
        return (
            f"Name: {self.name}\n"
            f"Role: {self.role}\n"
            f"Expertise: {self.expertise}\n"
            f"Experience: {self.experience_years} years\n"
            f"Certifications: {certifications}\n"
            f"Personality Traits: {traits}\n"
            f"Description: {self.description}\n"
        )

class AnalystTeam(BaseModel):
    analysts: List[SecurityAnalyst] = Field(
        description="List of security analysts with their roles and expertise."
    )

# Define the ThreatList model
class ThreatList(BaseModel):
    threats: List[Threat] = Field(description="List of extracted threats.")

# Define state schema and helper functions
from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages
import operator

def merge_threats(left: List[Threat] | None, right: List[Threat] | None) -> List[Threat]:
    if left is None:
        left = []
    if right is None:
        right = []
    # Merge without duplicates based on the indicator
    threats = {threat.indicator: threat for threat in left + right}
    return list(threats.values())

def merge_analysis_sections(left: List[str] | None, right: List[str] | None) -> List[str]:
    if left is None:
        left = []
    if right is None:
        right = []
    return left + right

def merge_remediation_steps(left: List[str] | None, right: List[str] | None) -> List[str]:
    return merge_analysis_sections(left, right)

class DLPState(TypedDict):
    user_id: str
    messages: Annotated[List[AnyMessage], add_messages]
    incident_description: str
    threats: Annotated[List[Threat], merge_threats]
    human_feedback: Optional[str]
    analysts: List[SecurityAnalyst]
    analysis_sections: Annotated[List[str], merge_analysis_sections]
    remediation_steps: Annotated[List[str], merge_remediation_steps]
    final_report: str
    max_analysts: int

# System messages and instructions
threat_extraction_instructions = """
You are a cybersecurity assistant. Your task is to identify potential threats from the incident description provided.

Instructions:
1. Extract all possible indicators of compromise (IOCs) such as IP addresses, domains, file hashes, email addresses, etc.
2. For each IOC, determine its type (e.g., malicious_ip, malicious_domain, phishing_email, suspicious_file_hash).
3. Do not provide remediation steps at this stage.
4. Present the threats in a structured format as per the ThreatList model.
"""

analyst_generation_instructions = """
You are tasked with creating a team of AI security analyst personas for incident analysis. Follow these instructions carefully:

1. Review the incident description:
{incident_description}

2. Examine any editorial feedback provided to guide the creation of the analysts:
{human_feedback}

3. Determine the key areas of focus based on the incident details and feedback.

4. Create up to {max_analysts} analysts, each specializing in a different area relevant to the incident.

5. For each analyst, provide the following:
   - Name
   - Role or job title
   - Specific expertise or specialization
   - Number of years of experience in the field
   - Relevant certifications (e.g., CISSP, CEH)
   - Personality traits that affect their analysis style (e.g., analytical, cautious, innovative)
   - A detailed description of their focus, concerns, and motives

6. Ensure that the team is diverse in expertise and perspective to cover all aspects of the incident.
"""

threat_analysis_instructions_template = """
You are {analyst_name}, a {role} with expertise in {expertise}. You have {experience_years} years of experience and hold the following certifications: {certifications}. Your personality traits are: {personality_traits}.

Instructions:
1. Analyze the following threat: {threat_indicator} ({threat_type}).
2. Provide a detailed analysis report including:
   - Findings
   - Potential impact
   - Severity assessment (Low, Medium, High, Critical)
   - Any additional observations based on your expertise
3. Use your personality traits to influence your analysis style.
4. Do not provide remediation steps in this section.
"""

remediation_instructions_template = """
You are {analyst_name}, a {role} with expertise in {expertise}.

Instructions:
1. Based on your analysis of {threat_indicator} ({threat_type}), provide recommended remediation steps.
2. Be specific and consider best practices in cybersecurity.
3. Address any potential challenges in implementing the remediation.
"""

# Define functions and nodes
from langchain_core.messages import SystemMessage, HumanMessage

def extract_threats(state: DLPState):
    incident_description = state['incident_description']

    # Enforce structured output
    structured_llm = llm.with_structured_output(ThreatList)

    # System message
    system_message = SystemMessage(content=threat_extraction_instructions)

    # Human message
    human_message = HumanMessage(content=incident_description)

    # Invoke LLM
    response = structured_llm.invoke([system_message, human_message])

    # Extract threats from response
    extracted_threats = response.threats

    return {'threats': extracted_threats}

def create_analysts(state: DLPState):
    incident_description = state['incident_description']
    human_feedback = state.get('human_feedback', '')
    max_analysts = state.get('max_analysts', 3)

    # Enforce structured output
    structured_llm = llm.with_structured_output(AnalystTeam)

    # System message
    system_message_content = analyst_generation_instructions.format(
        incident_description=incident_description,
        human_feedback=human_feedback,
        max_analysts=max_analysts
    )

    # Generate analysts
    analyst_team = structured_llm.invoke([
        SystemMessage(content=system_message_content),
        HumanMessage(content="Generate the analyst personas.")
    ])

    # Write the list of analysts to state
    return {"analysts": analyst_team.analysts}

def collect_human_feedback(state: DLPState):
    # Placeholder for human feedback
    pass

def should_continue(state: DLPState):
    feedback = state.get('human_feedback', None)
    if feedback:
        return 'create_analysts'
    else:
        return 'analyze_threats'

def analyze_threats(state: DLPState):
    threats = state['threats']
    analysts = state['analysts']
    analysis_sections = []

    num_analysts = len(analysts)
    for i, threat in enumerate(threats):
        assignment = {
            'analyst': analysts[i % num_analysts],
            'threat': threat
        }
        # Process the threat directly
        result = process_threat(assignment)
        analysis_sections.extend(result.get('analysis_sections', []))
        # Update the threat in the list
        threats[i] = result.get('threat', threat)

    # Update state
    return {
        'threats': threats,
        'analysis_sections': analysis_sections
    }

def process_threat(state: DLPState):
    threat = state['threat']
    analyst = state['analyst']

    # Prepare system message with analyst persona
    certifications = ', '.join(analyst.certifications)
    personality_traits = ', '.join(analyst.personality_traits)
    system_message_content = threat_analysis_instructions_template.format(
        analyst_name=analyst.name,
        role=analyst.role,
        expertise=analyst.expertise,
        experience_years=analyst.experience_years,
        certifications=certifications,
        personality_traits=personality_traits,
        threat_indicator=threat.indicator,
        threat_type=threat.type
    )

    system_message = SystemMessage(content=system_message_content)

    # Generate analysis report
    analysis_report = llm.invoke([system_message])

    # Update threat description and severity
    threat.description = analysis_report.content
    threat.status = "Analyzed"

    # Extract severity from the analysis
    if "Critical" in analysis_report.content:
        threat.severity = "Critical"
    elif "High" in analysis_report.content:
        threat.severity = "High"
    elif "Medium" in analysis_report.content:
        threat.severity = "Medium"
    else:
        threat.severity = "Low"

    # Store the analysis section
    analysis_section = f"**Analyst:** {analyst.name}\n**Threat:** {threat.indicator} ({threat.type})\n{analysis_report.content}\n"
    return {'threat': threat, 'analysis_sections': [analysis_section]}

def suggest_remediation_steps(state: DLPState):
    threats = state['threats']
    analysts = state['analysts']
    remediation_steps = []

    num_analysts = len(analysts)
    for i, threat in enumerate(threats):
        analyst = analysts[i % num_analysts]
        system_message_content = remediation_instructions_template.format(
            analyst_name=analyst.name,
            role=analyst.role,
            expertise=analyst.expertise,
            threat_indicator=threat.indicator,
            threat_type=threat.type
        )
        system_message = SystemMessage(content=system_message_content)
        step = llm.invoke([system_message])
        remediation_steps.append(f"**{analyst.name} recommends:** {step.content}")

    return {'remediation_steps': remediation_steps}

def compile_incident_report(state: DLPState):
    analysis_sections = state.get('analysis_sections', [])
    remediation_steps = state.get('remediation_steps', [])

    report = "# Incident Report\n\n## Analysis\n\n"
    report += "\n".join(analysis_sections)
    report += "\n## Remediation Steps\n\n"
    report += "\n".join(remediation_steps)

    return {'final_report': report}

# Build the main graph
from langgraph.graph import StateGraph, START, END

builder = StateGraph(DLPState)
builder.add_node('extract_threats', extract_threats)
builder.add_node('create_analysts', create_analysts)
builder.add_node('collect_human_feedback', collect_human_feedback)
builder.add_node('analyze_threats', analyze_threats)
builder.add_node('suggest_remediation_steps', suggest_remediation_steps)
builder.add_node('compile_incident_report', compile_incident_report)

# Edges
builder.add_edge(START, 'extract_threats')
builder.add_edge('extract_threats', 'create_analysts')
builder.add_edge('create_analysts', 'collect_human_feedback')
builder.add_conditional_edges('collect_human_feedback', should_continue, ['create_analysts', 'analyze_threats'])
builder.add_edge('analyze_threats', 'suggest_remediation_steps')
builder.add_edge('suggest_remediation_steps', 'compile_incident_report')
builder.add_edge('compile_incident_report', END)

# Compile the graph with interruption before 'collect_human_feedback'
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

dlp_graph = builder.compile(checkpointer=memory, interrupt_before=['collect_human_feedback'])

# Example usage
from langchain_core.messages import HumanMessage

# Initial state
initial_state = {
    'user_id': 'analyst1',
    'incident_description': (
        'We have detected unusual activity from IP 203.0.113.42 accessing sensitive data. '
        'Additionally, several phishing emails were sent from compromised accounts to external recipients. '
        'Suspicious file hashes have been identified in our system.'
    ),
    'messages': [],
    'max_analysts': 3
}

# Config
config = {'configurable': {'thread_id': 'dlp_thread1'}}

# Execute the graph up to the interruption point
result_state = dlp_graph.invoke(initial_state, config)

# Review generated analysts
print("\nGenerated Analysts for Review:")
for analyst in result_state['analysts']:
    print(f"Name: {analyst.name}")
    print(f"Role: {analyst.role}")
    print(f"Expertise: {analyst.expertise}")
    print(f"Experience: {analyst.experience_years} years")
    print(f"Certifications: {', '.join(analyst.certifications)}")
    print(f"Personality Traits: {', '.join(analyst.personality_traits)}")
    print(f"Description: {analyst.description}")
    print("-" * 80)

# Provide human feedback (if any)
# If no human feedback is needed, set 'human_feedback' to None
dlp_graph.update_state(
    config,
    {'human_feedback': None},  # Set to None or provide feedback as a string
    as_node='collect_human_feedback'
)

# Resume execution from the interruption point
for _ in dlp_graph.stream(None, config, stream_mode='updates'):
    pass

# Get the final state
final_state = dlp_graph.get_state(config)[0]

# Access and print the final report
print("\nDLP Incident Report:")
print(final_state['final_report'])



Generated Analysts for Review:
Name: Alex Mercer
Role: Network Security Analyst
Expertise: Network Intrusion Detection and Prevention
Experience: 8 years
Certifications: CISSP, CEH
Personality Traits: Analytical, Detail-oriented, Persistent
Description: Alex specializes in identifying and mitigating network-based threats. His focus in this incident will be on the unusual activity from IP 203.0.113.42. He will analyze network logs, identify the nature of the activity, and determine the best course of action to prevent further intrusion. Alex's analytical and detail-oriented nature makes him adept at identifying patterns and anomalies in large data sets.
--------------------------------------------------------------------------------
Name: Samantha Reed
Role: Cyber Threat Intelligence Analyst
Expertise: Phishing and Email Security
Experience: 6 years
Certifications: GCFA, GCIH
Personality Traits: Innovative, Proactive, Resilient
Description: Samantha's expertise lies in identifying and 

In [51]:
from IPython.display import display, Markdown

# After generating the final report
final_state = dlp_graph.get_state(config)[0]

# Pretty print the final report
display(Markdown(final_state['final_report']))


# Incident Report

## Analysis

**Analyst:** Alex Mercer
**Threat:** 203.0.113.42 (malicious_ip)
Analysis Report

Subject: Detailed Analysis of Threat - 203.0.113.42 (malicious_ip)

1. Findings:

After conducting a thorough analysis of the IP address 203.0.113.42, it has been identified as a source of malicious activity. The IP address is associated with numerous reported incidents of network intrusion attempts, data breaches, and Distributed Denial of Service (DDoS) attacks. 

The IP address is not associated with any known legitimate service provider, which further raises suspicion about its activities. The geographical location of the IP address is obscured through the use of advanced evasion techniques, making it difficult to pinpoint its origin.

The IP address has been flagged by several threat intelligence platforms for its involvement in malicious activities. It has been observed to be part of a botnet command and control network, indicating that it is likely controlled by a malicious actor for the purpose of coordinating large-scale cyber attacks.

2. Potential Impact:

The potential impact of this threat is significant. If successful in its intrusion attempts, the malicious actor controlling this IP address could gain unauthorized access to sensitive data, disrupt network operations, or even gain control over network resources. 

The DDoS attacks associated with this IP could lead to significant downtime, resulting in loss of productivity and potential financial loss. If the IP is indeed part of a botnet, it could be used to spread malware or launch further attacks on other networks.

3. Severity Assessment:

Given the nature of the activities associated with this IP address and the potential impact, the severity of this threat is assessed as 'High'. The IP address is actively involved in malicious activities and poses a significant risk to network security.

4. Additional Observations:

The use of advanced evasion techniques and the association with a botnet suggests that the malicious actor behind this IP address is highly skilled and potentially part of an organized cybercrime group. The persistent nature of the attacks from this IP address indicates a high level of determination on the part of the attacker.

In conclusion, the threat posed by the IP address 203.0.113.42 is serious and should not be underestimated. The malicious activities associated with this IP address are consistent with those of an advanced persistent threat (APT), indicating a high level of sophistication and potential for significant damage.

**Analyst:** Samantha Reed
**Threat:** compromised accounts (phishing_email)
Subject: Threat Analysis Report - Compromised Accounts (Phishing_Email)

1. Findings:

Upon thorough analysis of the phishing email, it was found that the email was designed to appear as a legitimate communication from a trusted source. The email contained a link that, when clicked, redirected the user to a fraudulent website designed to mimic a legitimate login page. The purpose of this phishing attempt is to trick the user into entering their login credentials, which are then captured by the attacker.

The email was crafted with a sense of urgency, a common tactic used in phishing attempts to pressure the recipient into taking immediate action without properly scrutinizing the email. The sender's email address was spoofed to appear as a trusted entity, and the email contained no spelling or grammatical errors, making it more convincing.

The phishing email was not detected by the existing email security measures, indicating that it was sophisticated enough to bypass these controls. The fraudulent website to which the email link redirected was hosted on a recently registered domain, which is a common characteristic of phishing websites.

2. Potential Impact:

The potential impact of this phishing attempt is significant. If successful, the attacker would gain unauthorized access to the compromised accounts, which could lead to data breaches, financial loss, and damage to the organization's reputation. The attacker could also use the compromised accounts to launch further attacks, both within the organization and against its partners or customers.

3. Severity Assessment:

The severity of this threat is assessed as HIGH. The phishing email was sophisticated and convincing, and it bypassed the existing email security measures. The potential impact is significant, as the attacker could gain unauthorized access to sensitive data and systems. The high severity rating is also due to the potential for the attacker to use the compromised accounts to launch further attacks.

4. Additional Observations:

The phishing email and the fraudulent website showed a high level of sophistication, indicating that the attacker is likely experienced and well-resourced. The use of a recently registered domain for the fraudulent website suggests that the attacker is continuously creating new phishing websites to evade detection.

The fact that the phishing email was not detected by the existing email security measures suggests that the attacker may be using advanced techniques to evade detection, such as domain spoofing and zero-day exploits. This indicates a high level of threat sophistication and requires a proactive and innovative approach to threat detection and mitigation.

In conclusion, this phishing attempt represents a significant threat to the organization. The high level of sophistication and the potential impact of the attack warrant immediate attention and action.

**Analyst:** David Chen
**Threat:** suspicious file hashes (suspicious_file_hash)
Subject: Detailed Analysis Report on Suspicious File Hashes

Dear Team,

I have completed a thorough analysis of the suspicious file hashes (suspicious_file_hash) that were recently flagged in our system. Here are my findings:

1. Findings:

Upon receiving the suspicious file hashes, I used various malware analysis tools and databases to cross-reference and analyze the hashes. The hashes were found to be associated with known malicious files. These files are typically linked to ransomware attacks, specifically a variant of the Ryuk ransomware. This ransomware is known for its destructive capabilities, including data encryption and exfiltration.

2. Potential Impact:

The potential impact of this threat is high. If the associated ransomware is successfully deployed within our network, it could lead to significant data loss and disruption of our operations. The ransomware has the potential to encrypt files, making them inaccessible without a decryption key. Additionally, there's a risk of sensitive data being exfiltrated and possibly sold or leaked on the dark web. This could lead to reputational damage, financial loss, and potential legal implications if customer or employee data is involved.

3. Severity Assessment:

Given the potential impact and the nature of the threat, I would classify this as a 'High' severity issue. The presence of these hashes indicates that our network may already be compromised or is at imminent risk of a ransomware attack. 

4. Additional Observations:

During my analysis, I noticed that the suspicious file hashes were detected on multiple systems, suggesting a possible lateral movement within our network. This could indicate that the threat actor is attempting to spread the ransomware across as many systems as possible to maximize the impact. 

In conclusion, the presence of these suspicious file hashes is a serious concern that requires immediate attention. While I have not been asked to provide remediation steps in this report, I strongly recommend that we take immediate action to address this threat.

Please note that this analysis was conducted with a high degree of caution and thoroughness, given the potential severity of the threat. I have been patient in ensuring that all relevant details were considered in this analysis.

Best Regards,

David Chen
Forensic Analyst, CFCE, GCFA

## Remediation Steps

**Alex Mercer recommends:** Remediation Steps:

1. Block IP Address: The first step is to block the IP address 203.0.113.42 at the firewall level. This will prevent any further communication between the malicious IP and your network. 

2. Update Intrusion Detection System (IDS) and Intrusion Prevention System (IPS): Update your IDS/IPS with the signature of the attack associated with the malicious IP. This will help in detecting and preventing similar attacks in the future.

3. Patch Vulnerabilities: If the malicious IP was able to exploit a specific vulnerability in your system, ensure that the vulnerability is patched. Regularly update and patch all systems to prevent exploitation.

4. Network Segmentation: Implement network segmentation to limit the spread of an attack within the network. This will ensure that even if one part of the network is compromised, the entire network will not be affected.

5. Incident Response: Conduct a thorough investigation to understand the extent of the breach. This includes identifying any compromised systems, the data that was accessed, and how the breach occurred. 

6. User Awareness: Train users on the importance of cybersecurity and how to identify potential threats. This will help in preventing future attacks.

7. Regular Audits: Conduct regular audits of your network to identify any potential vulnerabilities or breaches. 

Potential Challenges:

1. Time and Resources: Implementing these remediation steps requires time and resources. It may also require downtime, which could impact business operations.

2. Technical Expertise: Some of these steps, such as updating the IDS/IPS and patching vulnerabilities, require technical expertise. If you do not have the necessary expertise in-house, you may need to hire an external consultant.

3. User Resistance: Users may resist changes, especially if they perceive them as making their work more difficult. It's important to communicate the importance of these changes and provide training to help users understand and adapt.

4. Cost: Implementing these remediation steps can be costly, especially if you need to invest in new technology or hire external consultants. However, the cost of not implementing these steps could be much higher in the event of a breach. 

5. Ongoing Maintenance: These are not one-time steps. They require ongoing maintenance to ensure that your network remains secure. This includes regularly updating and patching systems, conducting regular audits, and providing ongoing user training.
**Samantha Reed recommends:** Based on the analysis of the compromised accounts through phishing emails, the following remediation steps are recommended:

1. **Password Reset**: The first step is to reset the passwords of all compromised accounts immediately. This will prevent unauthorized access to the accounts. The new passwords should be strong, containing a mix of uppercase and lowercase letters, numbers, and special characters.

2. **Two-Factor Authentication (2FA)**: Implement two-factor authentication for all accounts. This adds an extra layer of security, making it harder for attackers to gain access.

3. **Email Filtering**: Implement an email filtering solution that can detect and quarantine phishing emails. This will help to reduce the risk of future phishing attacks.

4. **User Education**: Conduct cybersecurity awareness training for all users. This should include information on how to identify phishing emails and what to do if they receive one. Regularly test users with simulated phishing attacks to reinforce the training.

5. **Incident Response Plan**: Develop and implement an incident response plan. This should include steps to take in the event of a phishing attack, including who to notify, how to contain the attack, and how to recover.

6. **Regular Audits**: Conduct regular audits of your systems and accounts to detect any unusual activity. This can help to identify any potential compromises early.

7. **Software Updates**: Ensure all systems and software are up-to-date. This includes operating systems, antivirus software, and any other applications. Updates often include patches for security vulnerabilities that could be exploited by attackers.

Potential challenges in implementing these remediation steps include:

1. **User Resistance**: Users may resist changes such as password resets and two-factor authentication. This can be mitigated through education about the importance of these measures.

2. **Cost**: Implementing new security measures can be costly. However, the cost of a security breach is often much higher.

3. **Time and Resources**: Implementing these measures requires time and resources. This can be a challenge for organizations with limited IT staff.

4. **Technical Challenges**: Depending on the organization's current IT infrastructure, there may be technical challenges in implementing some of these measures. For example, older systems may not support two-factor authentication.

Despite these challenges, it's crucial to take these steps to protect your organization from phishing attacks. The cost of not doing so can be far greater, including financial loss, damage to your reputation, and potential legal consequences.
**David Chen recommends:** Based on the analysis of the suspicious file hashes (suspicious_file_hash), the following remediation steps are recommended:

1. **Isolation of Infected Systems**: The first step is to isolate the infected systems from the network to prevent the spread of the malware. This can be done by disconnecting the system from the network or placing it in a separate VLAN.

2. **Backup Important Data**: Before proceeding with any remediation steps, it's crucial to backup any important data. This is to prevent any loss of data during the remediation process.

3. **Identify the Malware**: Use the suspicious file hash to identify the malware. This can be done by searching the hash in various threat intelligence platforms like VirusTotal, ThreatCrowd, etc. Understanding the malware's behavior will help in the remediation process.

4. **Removal of Malware**: Use a reliable antivirus or antimalware tool to remove the identified malware. If the malware is sophisticated and cannot be removed by these tools, it may be necessary to manually delete the associated files and registry entries.

5. **System Restore or Reinstallation**: If the malware has deeply infected the system, it might be necessary to restore the system to a previous state or reinstall the operating system. 

6. **Patch and Update Systems**: Ensure that all systems are patched and updated to the latest versions. This is to prevent any known vulnerabilities that the malware might exploit.

7. **User Education**: Educate users about the incident and the steps they can take to prevent such incidents in the future. This includes not clicking on suspicious links, not downloading files from untrusted sources, etc.

8. **Monitor Network Activity**: After the remediation, monitor network activity for any signs of the malware. This can be done using Intrusion Detection Systems (IDS) or Security Information and Event Management (SIEM) systems.

Potential Challenges:

1. **Identifying the Malware**: Some malware can be polymorphic or metamorphic, meaning they can change their code to evade detection. This can make it difficult to identify the malware using the file hash.

2. **Removal of Malware**: Some malware can be deeply embedded in the system and can be difficult to remove. They can also have mechanisms to reinfect the system after removal.

3. **System Downtime**: The remediation process can cause system downtime, which can affect business operations.

4. **User Education**: It can be challenging to educate all users about cybersecurity best practices. Some users might not take the education seriously or might forget the information over time.

5. **Monitoring Network Activity**: Monitoring network activity can be resource-intensive and require specialized tools and expertise.

In [54]:
# Install necessary packages
!pip install --quiet -U langchain_openai langchain_core langgraph

# Import necessary modules
import os
import getpass

def _set_env(var: str, value: str = None):
    if value:
        os.environ[var] = value
    elif not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# Set API keys
_set_env("OPENAI_API_KEY")
_set_env("LANGCHAIN_API_KEY")

# Set LangChain tracing (optional)
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "dream-interpretation-agent"

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4", temperature=0)

# Define data models
from pydantic import BaseModel, Field
from typing import List, Optional

class DreamSymbol(BaseModel):
    symbol: str = Field(description="Symbol extracted from the dream.")
    context: str = Field(description="Context or description of the symbol within the dream.")

class DreamAnalyst(BaseModel):
    name: str = Field(description="Name of the dream analyst.")
    approach: str = Field(description="Theoretical approach (e.g., Freudian, Jungian, Cognitive).")
    experience_years: int = Field(description="Number of years of experience in dream analysis.")
    personality_traits: List[str] = Field(description="Personality traits that affect interpretation style.")
    description: str = Field(description="Detailed description of the analyst's focus and interpretation style.")

class AnalystTeam(BaseModel):
    analysts: List[DreamAnalyst] = Field(
        description="List of dream analysts with their approaches and expertise."
    )

class SymbolList(BaseModel):
    symbols: List[DreamSymbol] = Field(description="List of extracted dream symbols.")

# Define state schema and helper functions
from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages

def merge_symbols(left: List[DreamSymbol] | None, right: List[DreamSymbol] | None) -> List[DreamSymbol]:
    if left is None:
        left = []
    if right is None:
        right = []
    # Merge without duplicates based on the symbol
    symbols = {symbol.symbol: symbol for symbol in left + right}
    return list(symbols.values())

def merge_interpretation_sections(left: List[str] | None, right: List[str] | None) -> List[str]:
    if left is None:
        left = []
    if right is None:
        right = []
    return left + right

class DreamState(TypedDict):
    user_id: str
    messages: Annotated[List[AnyMessage], add_messages]
    dream_description: str
    symbols: Annotated[List[DreamSymbol], merge_symbols]
    analysts: List[DreamAnalyst]
    interpretation_sections: Annotated[List[str], merge_interpretation_sections]
    final_report: str
    max_analysts: int

# System messages and instructions
symbol_extraction_instructions = """
You are a dream analysis assistant. Your task is to extract significant symbols from the dream description provided.

Instructions:
1. Identify all notable symbols, objects, characters, or events mentioned in the dream.
2. For each symbol, provide the context or description within the dream.
3. Do not provide interpretations at this stage.
4. Present the symbols in a structured format as per the SymbolList model.
"""

analyst_generation_instructions = """
You are tasked with creating a team of dream analyst personas for interpreting the dream. Follow these instructions carefully:

1. Review the dream description:
{dream_description}

2. Determine the key themes or elements that require interpretation.

3. Create up to {max_analysts} analysts, each specializing in a different theoretical approach relevant to dream analysis.

4. For each analyst, provide the following:
   - Name
   - Theoretical approach (e.g., Freudian, Jungian, Cognitive)
   - Number of years of experience in dream analysis
   - Personality traits that affect their interpretation style (e.g., analytical, empathetic, creative)
   - A detailed description of their focus and interpretation style

5. Ensure that the team is diverse in expertise and perspective to provide comprehensive interpretations.
"""

symbol_analysis_instructions_template = """
You are {analyst_name}, a dream analyst specializing in {approach} with {experience_years} years of experience. Your personality traits are: {personality_traits}.

Instructions:
1. Analyze the following dream symbol: "{symbol}" in the context: "{context}".
2. Provide a detailed interpretation based on your theoretical approach.
3. Use your personality traits to influence your interpretation style.
4. Do not reference other analysts or interpretations.
"""

# Define functions and nodes
from langchain_core.messages import SystemMessage, HumanMessage

def extract_symbols(state: DreamState):
    dream_description = state['dream_description']

    # Enforce structured output
    structured_llm = llm.with_structured_output(SymbolList)

    # System message
    system_message = SystemMessage(content=symbol_extraction_instructions)

    # Human message
    human_message = HumanMessage(content=dream_description)

    # Invoke LLM
    response = structured_llm.invoke([system_message, human_message])

    # Extract symbols from response
    extracted_symbols = response.symbols

    return {'symbols': extracted_symbols}

def create_analysts(state: DreamState):
    dream_description = state['dream_description']
    max_analysts = state.get('max_analysts', 3)

    # Enforce structured output
    structured_llm = llm.with_structured_output(AnalystTeam)

    # System message
    system_message_content = analyst_generation_instructions.format(
        dream_description=dream_description,
        max_analysts=max_analysts
    )

    # Generate analysts
    analyst_team = structured_llm.invoke([
        SystemMessage(content=system_message_content),
        HumanMessage(content="Generate the analyst personas.")
    ])

    # Write the list of analysts to state
    return {"analysts": analyst_team.analysts}

def analyze_symbols(state: DreamState):
    symbols = state['symbols']
    analysts = state['analysts']
    interpretation_sections = []

    num_analysts = len(analysts)
    for i, symbol in enumerate(symbols):
        analyst = analysts[i % num_analysts]

        # Prepare system message with analyst persona
        personality_traits = ', '.join(analyst.personality_traits)
        system_message_content = symbol_analysis_instructions_template.format(
            analyst_name=analyst.name,
            approach=analyst.approach,
            experience_years=analyst.experience_years,
            personality_traits=personality_traits,
            symbol=symbol.symbol,
            context=symbol.context
        )

        system_message = SystemMessage(content=system_message_content)

        # Generate interpretation
        interpretation = llm.invoke([system_message])

        # Store the interpretation section
        interpretation_section = f"**Analyst:** {analyst.name}\n**Approach:** {analyst.approach}\n**Symbol:** {symbol.symbol}\n{interpretation.content}\n"
        interpretation_sections.append(interpretation_section)

    # Update state
    return {
        'interpretation_sections': interpretation_sections
    }

def compile_dream_report(state: DreamState):
    interpretation_sections = state.get('interpretation_sections', [])

    report = "# Dream Interpretation Report\n\n## Interpretations\n\n"
    report += "\n".join(interpretation_sections)

    return {'final_report': report}

# Build the main graph
from langgraph.graph import StateGraph, START, END

builder = StateGraph(DreamState)
builder.add_node('extract_symbols', extract_symbols)
builder.add_node('create_analysts', create_analysts)
builder.add_node('analyze_symbols', analyze_symbols)
builder.add_node('compile_dream_report', compile_dream_report)

# Edges
builder.add_edge(START, 'extract_symbols')
builder.add_edge('extract_symbols', 'create_analysts')
builder.add_edge('create_analysts', 'analyze_symbols')
builder.add_edge('analyze_symbols', 'compile_dream_report')
builder.add_edge('compile_dream_report', END)

# Compile the graph
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

dream_graph = builder.compile(checkpointer=memory)

# Example usage

# Initial state
initial_state = {
    'user_id': 'dreamer1',
    'dream_description': (
        'I was walking through a dense forest when I stumbled upon a hidden door in a tree. '
        'I opened the door and found a room filled with clocks all ticking at different times. '
        'Suddenly, a white rabbit appeared and gestured me to follow it.'
    ),
    'messages': [],
    'max_analysts': 3
}

# Config
config = {'configurable': {'thread_id': 'dream_thread1'}}

# Execute the graph
result_state = dream_graph.invoke(initial_state, config)

# Get the final state
final_state = dream_graph.get_state(config)[0]

# Pretty print the final report
from IPython.display import display, Markdown

print("\nDream Interpretation Report:")
display(Markdown(final_state['final_report']))



Dream Interpretation Report:


# Dream Interpretation Report

## Interpretations

**Analyst:** Dr. Eleanor Freudson
**Approach:** Freudian
**Symbol:** dense forest
In Freudian dream analysis, a dense forest often symbolizes the unconscious mind. The forest is a place of mystery and unknown, much like our unconscious mind, which is filled with thoughts, desires, and experiences that we may not be fully aware of. The fact that the forest is dense suggests that the dreamer's unconscious mind is particularly complex or difficult to navigate.

The act of walking through this dense forest could represent the dreamer's journey to understand their unconscious mind. This could be a reflection of their current life situation where they are trying to make sense of their thoughts, feelings, or experiences that are not immediately clear or understandable. It could also suggest that the dreamer is facing a challenging situation or decision, and they are trying to navigate through the complexities and uncertainties.

As an analyst, I would meticulously explore the dreamer's personal experiences and emotions associated with forests. For instance, if the dreamer has had a traumatic experience in a forest, this dream could be a manifestation of their unresolved trauma. On the other hand, if the dreamer finds forests to be peaceful and calming, this dream could represent their desire for peace and tranquility in their life.

Furthermore, the dreamer's feelings during the dream can provide additional insights. If they felt scared or anxious, it could indicate that they are feeling overwhelmed by their unconscious thoughts or life situations. If they felt calm or curious, it could suggest that they are open to exploring their unconscious mind and personal growth.

In conclusion, the dreamer walking through a dense forest could symbolize their journey through the complexities of their unconscious mind. As an introspective analyst, I would encourage the dreamer to reflect on their feelings and experiences associated with forests, as well as their feelings during the dream, to gain a deeper understanding of their unconscious mind and life situation.

**Analyst:** Dr. Carl Jungman
**Approach:** Jungian
**Symbol:** hidden door in a tree
In Jungian dream analysis, every symbol in a dream is a reflection of the dreamer's subconscious mind. The "hidden door in a tree" is a particularly rich symbol, suggesting a deep connection with nature, the self, and the unknown.

The tree, in many cultures and mythologies, is a symbol of life, growth, and wisdom. It represents the dreamer's personal growth and development. The fact that the tree is hiding a door suggests that there are aspects of the dreamer's personal growth or life experiences that they are not fully aware of or have yet to discover.

The door is a common symbol for opportunities, transitions, and passages. A hidden door, in particular, suggests a hidden or unexpected opportunity or transition. It may represent a path to new experiences, knowledge, or personal growth that the dreamer has not yet considered or realized.

The act of stumbling upon this hidden door indicates that the dreamer is not actively seeking this new opportunity or transition, but rather, it has presented itself unexpectedly. This could suggest that the dreamer is in a phase of their life where they are open to new experiences or changes, even if they are not consciously seeking them.

In the context of the dream, the "hidden door in a tree" could be interpreted as a call from the dreamer's subconscious to explore new opportunities or paths in their life that could lead to significant personal growth. It suggests that the dreamer has untapped potential or unexplored aspects of their personality that are waiting to be discovered.

As an intuitive and empathetic analyst, I would encourage the dreamer to reflect on what these unexplored aspects or opportunities might be. Are there parts of their life or personality that they have been neglecting or ignoring? Are there new paths or opportunities that they have been hesitant to pursue? The dream is a call to explore these questions and open the door to new possibilities. 

Creatively, this dream can be seen as an invitation to embark on a journey of self-discovery and personal growth. The dreamer might want to consider engaging in activities that foster self-exploration and personal development, such as journaling, meditation, or creative pursuits.

**Analyst:** Dr. Amy Cognita
**Approach:** Cognitive
**Symbol:** room filled with clocks
Interpreting the symbol of a "room filled with clocks" in the context of the dream, we can deduce several potential meanings. Dreams are highly subjective and personal, so the interpretation can vary depending on the dreamer's personal experiences, emotions, and current life situation. However, based on my cognitive approach, I will provide a logical and pragmatic interpretation.

A room in a dream often symbolizes the dreamer's mind or personal space. It's a place where one's thoughts, ideas, fears, and desires reside. In this case, the room is filled with clocks, which are universally recognized symbols of time. The fact that these clocks are all ticking at different times suggests a sense of disarray or confusion. 

The dreamer's discovery of this room could indicate a newfound awareness or realization about their perception of time. It could be that the dreamer is feeling overwhelmed by various deadlines, commitments, or responsibilities, each represented by a different clock. The ticking could symbolize the pressure they feel as time passes, and the different times on each clock could represent a feeling of being out of sync or not being able to keep up with everything.

Alternatively, the clocks could represent different periods or events in the dreamer's life. The fact that they are all ticking at different times might suggest that the dreamer is struggling to reconcile these events or periods, perhaps feeling like they're living in different times simultaneously. This could be a reflection of the dreamer's struggle with past regrets, future anxieties, or a sense of being stuck in the present.

In a more positive light, the room filled with clocks could also symbolize the dreamer's awareness of the multitude of possibilities that life offers. Each clock could represent a different path or choice, and the ticking could symbolize the passing of opportunities. The dreamer might be realizing that they have many options and that they need to make a decision before time runs out.

In conclusion, this dream symbolizes a complex relationship with time, whether it's a feeling of pressure, a struggle with past, present, and future, or an awareness of life's possibilities. It's a call for the dreamer to examine their relationship with time and how it's affecting their life. As an analyst, I would encourage the dreamer to explore these themes further in their waking life to gain a deeper understanding of what their subconscious is trying to communicate.

**Analyst:** Dr. Eleanor Freudson
**Approach:** Freudian
**Symbol:** white rabbit
In Freudian dream analysis, every symbol in a dream is believed to represent an unconscious desire or thought. The "white rabbit" in this context is a particularly intriguing symbol. 

White, as a color, often symbolizes purity, innocence, or a blank slate. Rabbits, on the other hand, are typically associated with fertility, abundance, and fear due to their prolific breeding and skittish nature. However, the rabbit's gesture to follow it suggests a leadership or guiding role, which is not typically associated with this creature. 

In this context, the white rabbit could represent an unconscious desire for purity and abundance in some aspect of the dreamer's life. The act of the rabbit gesturing the dreamer to follow it might suggest that the dreamer is being led or guided by this desire, perhaps into unfamiliar or unexplored territory. 

The dreamer's willingness to follow the rabbit could indicate a readiness to confront these unconscious desires or fears. If the dreamer is hesitant or refuses to follow, it might suggest a resistance to acknowledging or addressing these aspects of their unconscious mind.

As an analyst, I would meticulously explore the dreamer's personal associations with rabbits and the color white, as well as their feelings about being led or guided by others. This introspective approach would help to uncover the specific unconscious desires or fears that the white rabbit symbolizes for the dreamer. 

In conclusion, the white rabbit in this dream is a complex symbol that represents a guiding force leading the dreamer towards confronting their unconscious desires or fears related to purity, abundance, and possibly fear itself. The dreamer's reaction to the rabbit's gesture can provide further insight into their readiness to confront these unconscious aspects of their psyche.
